In [1]:
test = '''
contract get_fib_n(n : int){

    let x: int = 0;

    entry sum(n: int){
        let a: int = 2;
        let b: int = 3;
        if (a > b) {
            x = a + b;
        }
        else {
            x = a - b;
        }

    }
}
'''

In [2]:
from lexer.sly_lexer import TzScriptLexer, process_lexer_tokens 
from lexer.lex_token import Token
from parser.tzscript_grammar import TZSCRIPT_GRAMMAR,idx, num, typex, contract, ifx, elsex,equal, plus, minus, star, div,semi, colon, comma, dot, opar, cpar, ocur, ccur,let, func,entry,lessthanequal,greaterthanequal, iniquelaty, lessthan,greaterthan,equalequal, returnx
from parser.slr_parser import SLR1Parser, build_slr_ast

lexer = TzScriptLexer()
lexer_tokens = list(lexer.tokenize(test))
tokens = process_lexer_tokens(lexer_tokens)

parser = SLR1Parser(TZSCRIPT_GRAMMAR, verbose=True)

terminals = [token.token_type for token in tokens]
derivation = parser(terminals, True)

0 	 S' -> .<program>, 
	 <program> -> .contractid(<param-list>){<stat_list>},  

1 	 <program> -> contract.id(<param-list>){<stat_list>},  

2 	 <program> -> contractid.(<param-list>){<stat_list>},  

3 	 <param> -> .id:type, 
	 <param-list> -> .<param>, 
	 <param-list> -> .<param>,<param-list>, 
	 <program> -> contractid(.<param-list>){<stat_list>},  

4 	 <param-list> -> <param>.,<param-list>, 
	 <param-list> -> <param>.,  

5 	 <param> -> .id:type, 
	 <param-list> -> .<param>, 
	 <param-list> -> <param>,.<param-list>, 
	 <param-list> -> .<param>,<param-list>,  

6 	 <param-list> -> <param>,<param-list>.,  

7 	 <param> -> id.:type,  

8 	 <param> -> id:.type,  

9 	 <param> -> id:type.,  

10 	 <program> -> contractid(<param-list>.){<stat_list>},  

11 	 <program> -> contractid(<param-list>).{<stat_list>},  

12 	 <def-entry> -> .entryid(<param-list>){<stat_list>}, 
	 <if-stat> -> .if(<expr>){<stat_list>}, 
	 <stat> -> .<let-var>>, 
	 <while-stat> -> .while(<expr>){<stat_list>}, 
	 

In [3]:
productions, operations = derivation


In [4]:
ast = build_slr_ast(productions, operations, tokens)

In [5]:
from visitors.string_rep_visitor import FormatVisitor
format = FormatVisitor()
print(format.visit(ast))

\__ProgramNode: contract node.idx(n : int) [<stat>; ... <stat>;]
	\__AttrDeclarationNode: n : int
		\__VarDeclarationNode: let x = <expr> : int
			\__ ConstantNumNode: 0
		\__EntryDeclarationNode: Entry sum(n : int)
			\__AttrDeclarationNode: n : int
				\__VarDeclarationNode: let a = <expr> : int
					\__ ConstantNumNode: 2
				\__VarDeclarationNode: let b = <expr> : int
					\__ ConstantNumNode: 3
				\__IfNode: if <expr> then [<stat>; ... <stat>;]
					\__ GreaterThanNode
						\__ VariableNode: a
						\__ VariableNode: b
					\__VarCallNode: x = <expr>
						\__ PlusNode
							\__ VariableNode: a
							\__ VariableNode: b
				\__ElseNode: else [<stat>; ... <stat>;]
					\__VarCallNode: x = <expr>
						\__ MinusNode
							\__ VariableNode: a
							\__ VariableNode: b


In [6]:
from visitors.high_level_ir_generator_visitor import TzScriptToHighLevelIrVisitor
high_level_ir = TzScriptToHighLevelIrVisitor()

In [7]:
from visitors.hl_string_repre import HLReprVisitor
hl_repr = HLReprVisitor()
ir = high_level_ir.visit(ast)

mirame
mirame
statements in entry:  <parser.tzscript_ast.VarDeclarationNode object at 0x7f11d6fb6190>
statements in entry:  <parser.tzscript_ast.VarDeclarationNode object at 0x7f11d6fb6550>
statements in entry:  <parser.tzscript_ast.IfNode object at 0x7f11d6fb4150>
here
lo creeeeee
nodos donde suma
here
nodos donde suma
statements in entry:  <parser.tzscript_ast.ElseNode object at 0x7f11d6fb4490>
here
nodos donde suma


In [8]:
print(hl_repr.visit(ir))

los:  [<intermediate_ast.high_level_ir_ast.IfEntryNode object at 0x7f11d6d6fa50>]
mmm [<intermediate_ast.high_level_ir_ast.PushValueNode object at 0x7f11d6d6f990>, <intermediate_ast.high_level_ir_ast.PushVariableNode object at 0x7f11d6fd2710>, <intermediate_ast.high_level_ir_ast.PushValueNode object at 0x7f11d6d6f9d0>, <intermediate_ast.high_level_ir_ast.PushVariableNode object at 0x7f11d6d6f550>, <intermediate_ast.high_level_ir_ast.IfStatementNode object at 0x7f11d6d74450>]
operation node:  <intermediate_ast.high_level_ir_ast.GreaterThanNode object at 0x7f11d6d6fd90>
operation node:  <intermediate_ast.high_level_ir_ast.PlusNode object at 0x7f11d6d74210>
operation node:  <intermediate_ast.high_level_ir_ast.MinusNode object at 0x7f11d6d74250>
contract { entrypoint{...} storage{...} code{...}} 
	entrypoint  { <id,params> , <id,params> , ... , <id,params> }
		\__EntryPointDeclarationNode: sum ([<parser.tzscript_ast.AttrDeclarationNode object at 0x7f11d6ee5290>])
	storage { <id,type> , <id

In [9]:
for x in ir.entrypoints.entrypoint_list:
    print(x.id)

sum


In [10]:
from visitors.michelson_generator_visitor import MichelsonGenerator
visit_generator = MichelsonGenerator()



In [11]:
code = visit_generator.visit(ir)

reference counter:  {'a': 3, 'b': 3, 'x': 2}
push value node:  <class 'str'> <class 'str'>
push value node:  <class 'str'> <class 'str'>
estado de counnter {'a': 3, 'b': 3, 'x': 2}
estado de counnter {'a': 2, 'b': 3, 'x': 2}
code parameter int %sum;
storage int;
code {
UNPAIR;
PUSH nat 2;
PUSH nat 3;
DIG 1;
DIG 1;
DUP;
DIG 2;
SWAP;

first 3 num
second 2 num
estado de counnter {'a': 2, 'b': 2, 'x': 2}
estado de counnter {'a': 1, 'b': 2, 'x': 2}
code parameter int %sum;
storage int;
code {
UNPAIR;
PUSH nat 2;
PUSH nat 3;
DIG 1;
DIG 1;
DUP;
DIG 2;
SWAP;
GT;
IF
then {
DIG 1;
DUP;
DIG 2;
SWAP;

first 3 num
second 0 int
current stack,  [<visitors.michelson_generator_visitor.StackValue object at 0x7f11d6d76bd0>, <visitors.michelson_generator_visitor.StackValue object at 0x7f11d6d77390>]
current-code:  parameter int %sum;
storage int;
code {
UNPAIR;
PUSH nat 2;
PUSH nat 3;
DIG 1;
DIG 1;
DUP;
DIG 2;
SWAP;
GT;
IF
then {
DIG 1;
DUP;
DIG 2;
SWAP;
ADD;

estado de counnter {'a': 1, 'b': 1, 'x': 1}
e